# Step 1: Install Requirements

In [1]:
#clone YOLOv5 and
# from google.colab import drive
# drive.mount('/content/drive', force_remount=True)
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
%pip install -qr requirements.txt # install dependencies
%pip install -q roboflow

import torch
import os
from IPython.display import Image, clear_output  # to display images

print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

fatal: destination path 'yolov5' already exists and is not an empty directory.


/home/mizanul/Documents/code/document_validator/yolov5
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Setup complete. Using torch 2.0.1+cu117 (NVIDIA GeForce GTX 1650)


In [2]:
# !pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="fIx2dux10SnLtQWgAYdJ")
project = rf.workspace("document-validator").project("yolo_validator")
version = project.version(2)
dataset = version.download("yolov5")
                

loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to yolo_validator-2 in yolov5pytorch:: 100%|██████████| 1208/1208 [00:00<00:00, 5593.26it/s]


# Step 3: Train Our Custom YOLOv5 model

Here, we are able to pass a number of arguments:
- **img:** define input image size
- **batch:** determine batch size
- **epochs:** define the number of training epochs. (Note: often, 3000+ are common here!)
- **data:** Our dataset locaiton is saved in the `dataset.location`
- **weights:** specify a path to weights to start transfer learning from. Here we choose the generic COCO pretrained checkpoint.
- **cache:** cache images for faster training

In [3]:
# set up environment
# os.environ["DATASET_DIRECTORY"] = "/yolo_validator-2"

In [4]:
print({dataset.location})

{'/home/mizanul/Documents/code/document_validator/yolov5/yolo_validator-2'}


In [5]:
# %cd yolov5

In [12]:
!python3 train.py --img 640 --batch 16 --epochs 2 --data /home/mizanul/Documents/code/document_validator/yolov5/yolo_validator-2/data.yaml --weights yolov5s.pt --cache

train: weights=yolov5s.pt, cfg=, data=/home/mizanul/Documents/code/document_validator/yolov5/yolo_validator-2/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=2, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data/hyps, resume_evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest, ndjson_console=False, ndjson_file=False
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v7.0-361-gc5ffbbf1 Python-3.10.12 torch-2.0.1+cu117 CUDA:0 (NVIDIA GeForce GTX 1650, 3904MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs

# Evaluate Custom YOLOv5 Detector Performance
Training losses and performance metrics are saved to Tensorboard and also to a logfile.

If you are new to these metrics, the one you want to focus on is `mAP_0.5` - learn more about mean average precision [here](https://blog.roboflow.com/mean-average-precision/).

In [7]:
# Start tensorboard
# Launch after you have started training
# logs save in the folder "runs"
%load_ext tensorboard
%tensorboard --logdir runs

Launching TensorBoard...

In [8]:
cd /home/mizanul/Documents/code-for-ta/yolov5

[Errno 2] No such file or directory: '/home/mizanul/Documents/code-for-ta/yolov5'
/home/mizanul/Documents/code/document_validator/yolov5


#Run Inference  With Trained Weights
Run inference with a pretrained checkpoint on contents of `test/images` folder downloaded from Roboflow.

In [9]:
!python detect.py --weights runs/train/exp/weights/best.pt --img 416 --conf 0.1 --source Documents/code-for-ta/Testing_Video/test_1.mp4

/bin/bash: line 1: python: command not found


In [10]:
#display inference on ALL test images

import glob
from IPython.display import Image, display

for imageName in glob.glob('/content/yolov5/runs/detect/exp/*.jpg'): #assuming JPG
    display(Image(filename=imageName))
    print("\n")

# Conclusion and Next Steps

Congratulations! You've trained a custom YOLOv5 model to recognize your custom objects.

To improve you model's performance, we recommend first interating on your datasets coverage and quality. See this guide for [model performance improvement](https://github.com/ultralytics/yolov5/wiki/Tips-for-Best-Training-Results).

To deploy your model to an application, see this guide on [exporting your model to deployment destinations](https://github.com/ultralytics/yolov5/issues/251).

Once your model is in production, you will want to continually iterate and improve on your dataset and model via [active learning](https://blog.roboflow.com/what-is-active-learning/).

In [11]:
#export your model's weights for future use
from google.colab import files
files.download('./runs/train/exp/weights/best.pt')

ModuleNotFoundError: No module named 'google.colab'